<a href="https://colab.research.google.com/github/sanjiv1980/ML/blob/main/NameTrainingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-02-18 18:01:03--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-02-18 18:01:04--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-02-18 18:01:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp

In [ ]:
spark = sparknlp.start()
spark = sparknlp.start(gpu=True)
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.1
Apache Spark version:  3.0.3


In [ ]:
!wget -q training.csv https://github.com/sanjiv1980/ML.git/training.csv

In [ ]:
with open("training.csv") as f:
    c=f.read()

print (c[:200])







<!DOCTYPE html>
<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.github


In [ ]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './training.csv')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               <html|[[document, 0, 4,...|[[document, 0, 4,...|[[token, 0, 4, <h...|[[pos, 0, 4, lang...|[[named_entity, 0...|
|               <link|[[document, 0, 4,...|[[document, 0, 4,...|[[token, 0, 4, <l...|[[pos, 0, 4, rel=...|[[named_entity, 0...|
|<link <link <link...|[[document, 0, 22...|[[document, 0, 22...|[[token, 0, 4, <l...|[[pos, 0, 4, cros...|[[named_entity, 0...|
|               <link|[[document, 0, 4,...|[[document, 0, 4,...|[[token, 0, 4, <l...|[[pos, 0, 4, cros...|[[named_entity, 0...|
|<script <script <...|[[document, 0, 91...|[[document, 0, 91...|[[token, 0, 6, <s...|[[pos, 0, 6, cros..

In [ ]:
training_data.count()

278

In [ ]:
# we use BERT Tiny
bert_annotator = BertEmbeddings.pretrained('bert_base_cased', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(5)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  #.setTestDataset("test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

In [ ]:
ner_model=pipeline.fit(training_data)

IllegalArgumentException: ignored

In [ ]:
ner_model.stages[1].write().save('NER_bert_mytest2')

In [ ]:
from pyspark.ml import Pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)

loaded_ner_model = NerDLModel.load("NER_bert_mytest2")\
 .setInputCols(["sentence", "token", "bert"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter])

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
text = "My name is khan kumar"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = ner_prediction_pipeline.fit(prediction_data)
preds = prediction_model.transform(prediction_data)
preds.show()

In [ ]:
preds.select('ner_span.result').take(1)

In [ ]:
preds.select('token.result','ner.result','ner.metadata').show(truncate=False)

In [ ]:
import pyspark.sql.functions as F

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

In [ ]:
from sparknlp.base import LightPipeline
light_model = LightPipeline(prediction_model)
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/utils/ner_highlighter.py
import ner_highlighter

In [ ]:
text = "My name is khan kumar"

In [ ]:
result = light_model.fullAnnotate(text)[0]
ner_highlighter.chunk_highlighter(result, entity_column="ner_span")

In [ ]:
text=  "My name is kumar gaurav"
result = light_model.fullAnnotate(text)[0]
ner_highlighter.chunk_highlighter(result, entity_column="ner_span")